In [16]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [9]:
df = pd.read_csv('testing notebooks/data/augmented_data.csv')

In [10]:
input_col = ['Growth Temp (Celsius)', 'Metal_mmol (mmol)', 'Chalcogen_mmol (mmol)',
             'Amines_mmol (mmol)', 'CA_mmol (mmol)', 'Phosphines_mmol (mmol)', 
             'S_I_amount (g)', 'S_II_amount (g)', 'Time_min (min)', 
             'x0_cadmium acetate', 'x0_cadmium acetate dihydrate', 
             'x0_cadmium oxide', 'x0_cadmium stearate', 'x0_dimethylcadmium', 
             'x1_None', 'x1_benzoic acid', 'x1_dodecylphosphonic acid', 
             'x1_ethylphosphonic acid', 'x1_lauric acid', 
             'x1_myrstic acid', 'x1_oleic acid', 'x1_stearic acid',
             'x2_2-6-dimethylpyridine', 'x2_None', 'x2_aniline', 
             'x2_benzylamine', 'x2_dioctylamine/hexadecylamine',
             'x2_dodecylamine', 'x2_heptylamine', 'x2_hexadecylamine', 
             'x2_octadecylamine', 'x2_octylamine', 'x2_oleylamine', 
             'x2_pyridine', 'x2_trioctylamine', 'x3_None', 'x3_diphenylphosphine', 
             'x3_tributylphosphine', 'x3_trioctylphosphine', 
             'x3_triphenylphosphine', 'x4_None', 'x4_liquid parafin', 
             'x4_octadecene', 'x4_phenyl ether', 'x4_trioctylphosphine oxide', 
             'x5_None', 'x5_phosphinic acid', 'x5_trioctylphosphine oxide'
            ]

output_col = ['Diameter_nm','Absorbance max (nm)', 'PL max (nm)']

X = df[input_col]

y = df[output_col]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45, shuffle=True)

# Preliminary evaluatiopn of some regressions

Note: The MSE or r2 of the multiouput regression outcomes are the average of all MSE or r2 outputs. 

In [14]:
#This is used to roughly show which regression performs better.
# Testing Regressions:

REGRESSIONS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10,
                                       max_features=32,     
                                       random_state=44),
    "K-nn": KNeighborsRegressor(),                          
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
    "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
    "MultiO/P GBR" :MultiOutputRegressor(GradientBoostingRegressor(n_estimators=5)),
    "MultiO/P AdaB" :MultiOutputRegressor(AdaBoostRegressor(n_estimators=5))
            }
# r2 is used to evaluate the performance of all regressions.

r2_list = list()

for name, reg in REGRESSIONS.items():     
    reg.fit(X_train, y_train)                 
    y_pred = pd.DataFrame(reg.predict(X_test))
    
    print(name, '\n')
    
# This loop will show r2 for each outcome
    for column in range(0, 3):
        r2 = r2_score(y_test.iloc[:, column], y_pred.iloc[:, column])
        r2_list.append(r2)
    
    print(' R^2 for diameter is ', r2_list[0], '\n',
          'R^2 for Absorbance is ', r2_list[1], '\n',
          'R^2 for PL is ', r2_list[2], '\n', '\n', 
         )
    del r2_list[:] #reset the list for the next regression

Extra trees 

 R^2 for diameter is  0.6574989759034051 
 R^2 for Absorbance is  0.9084914764635887 
 R^2 for PL is  0.7127956509704536 
 

K-nn 

 R^2 for diameter is  0.4339480735015191 
 R^2 for Absorbance is  0.5219067936995085 
 R^2 for PL is  0.3585934604810823 
 

Linear regression 

 R^2 for diameter is  0.138806072261894 
 R^2 for Absorbance is  0.23583055000921482 
 R^2 for PL is  -0.3110746561026876 
 

Ridge 

 R^2 for diameter is  0.309922550724838 
 R^2 for Absorbance is  0.5217494772222664 
 R^2 for PL is  0.32610425909819574 
 

Lasso 

 R^2 for diameter is  -0.0012262257360822204 
 R^2 for Absorbance is  0.4673181908902928 
 R^2 for PL is  0.24697801344016912 
 

ElasticNet 

 R^2 for diameter is  0.0312198570884763 
 R^2 for Absorbance is  0.39595135476114374 
 R^2 for PL is  0.16136347077262214 
 

RandomForestRegressor 

 R^2 for diameter is  0.48607560083773804 
 R^2 for Absorbance is  0.7419770364392917 
 R^2 for PL is  0.5490578569971812 
 

Decision Tree Regresso

From the R2 values above, Extra Trees, Random Forest, Ridge, Decision Tree, and MultiO/P AdaB give the best predictions.

Now, we are going to optimize each of the the above regressions by grid searching the parameters

# Extra Trees

In [17]:
max_r2 = 0

max_i, max_j, max_k  = 0, 0, 0

for i in tqdm(range(1, 20)):
    for j in range(1, 10):
        for k in range(0, 80, 1):
            
            ET_reg = ExtraTreesRegressor(n_estimators=i,
                                         max_features=j,     
                                         random_state=k).fit(X_train, y_train)
            
            
            ET_y_pred = ET_reg.predict(X_test)
            
            ET_r2 = r2_score(y_test, pd.DataFrame(ET_y_pred))
            
            if (max_r2 < ET_r2):
                max_r2 = ET_r2
                max_i = i
                max_j = j
                max_k = k
            
print(max_r2, min_i, min_j, min_k)

100%|██████████| 19/19 [03:37<00:00, 11.46s/it]

0.8442759697637148 3 8 44


From the results above the parameters are applied to the model

In [18]:
ET_reg = ExtraTreesRegressor(n_estimators=3,
                            max_features=8,     
                            random_state=44).fit(X_train, y_train)
            
            
ET_y_pred = ET_reg.predict(X_test)

outputs = ('diameter: ', 'Abs: ', 'PL: ')

for i in range(0, 3):
    ET_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(ET_y_pred).loc[:, i])
    ET_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(ET_y_pred).loc[:, i])
    
    print(outputs[i], 'r2:', ET_r2, '; MSE:', ET_MSE )
    

diameter:  r2: 0.8113666509442224 ; MSE: 0.49611103917257676
Abs:  r2: 0.9397746426000377 ; MSE: 363.20520094562653
PL:  r2: 0.7816866157468844 ; MSE: 467.1041395883966


# Random Forest

In [ ]:
max_r2 = 0

max_i, max_j, max_k, max_m  = 0, 0, 0, 0

for i in tqdm(range(5, 15)):
    for j in range(5, 20):
        for k in range(10, 20):
            for m in range (40, 60):
                RF_reg = RandomForestRegressor(max_depth=i, 
                                                n_estimators=j, 
                                                max_features=k,
                                                random_state=m).fit(X_train, y_train)

                RF_y_pred = RF_reg.predict(X_test)

                RF_r2 = r2_score(y_test, pd.DataFrame(RF_y_pred))

                if (max_r2 < RF_r2):
                    max_r2 = RF_r2
                    max_i = i
                    max_j = j
                    max_k = k
                    max_m = m
            
print(max_r2, max_i, max_j, max_k, max_m)

From the results above (*take a long time to run*) the parameters are applied to the model

In [20]:
RF_reg = RandomForestRegressor(max_depth=13, 
                               n_estimators=5, 
                               max_features=14,
                               random_state=57).fit(X_train, y_train)
            
RF_y_pred = RF_reg.predict(X_test)

outputs = ('diameter: ', 'Abs : ', 'PL: ')
for i in range(0, 3):
    RF_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(RF_y_pred).loc[:, i])
    RF_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(RF_y_pred).loc[:, i])
    print(outputs[i], 'r2:', RF_r2, '; MSE:', RF_MSE )

diameter:  r2: 0.7509832185554824 ; MSE: 0.6549211729115839
Abs :  r2: 0.8919379340477296 ; MSE: 651.6973260638301
PL:  r2: 0.8144677642148114 ; MSE: 396.9654708017072


# Lasso

In [22]:
min_r2 = 0

min_i, min_j, min_k, min_m  = 0, 0, 0, 0

for i in tqdm(np.arange(0.1, 2.0, 0.02)):
    for j in range(1, 100):
        L_reg = Lasso(alpha=i, random_state=j).fit(X_train, y_train)
        L_y_pred = L_reg.predict(X_test)
        L_r2 = r2_score(y_test, pd.DataFrame(L_y_pred))
      
        if (min_r2 < L_r2):
                min_r2 = L_r2
                min_i = i
                min_j = j

            
print(min_r2, min_i, min_j)

100%|██████████| 95/95 [01:16<00:00,  1.24it/s]

0.4248557089497587 0.1 1


From the results above the parameters are applied to the model

In [40]:
L_reg = Lasso(alpha=0.1, random_state=1).fit(X_train, y_train)
            
L_y_pred = L_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    L_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(L_y_pred).loc[:, i])
    L_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(L_y_pred).loc[:, i])
    print(outputs[i], 'r2:', L_r2, '; MSE:', L_MSE )

diameter: r2: 0.2521760419982201 ; MSE: 1.9667981445460596
Abs: r2: 0.5747460874430704 ; MSE: 2564.607989578328
PL: r2: 0.4476449974079858 ; MSE: 1181.8208449095912


# Decision Trees

In [ ]:
max_r2 = 0

max_i, max_j, max_k, max_m  = 0, 0, 0, 0

for i in tqdm(range(5, 15)):
    for j in range(5, 15):
        for k in range(40, 60):
            for m in range (10, 50, 2):
                DT_reg = DecisionTreeRegressor(max_depth=i, 
                                               max_features=j,
                                               random_state=k, 
                                               max_leaf_nodes=m).fit(X_train, y_train)

                DT_y_pred = DT_reg.predict(X_test)

                DT_r2 = r2_score(y_test, pd.DataFrame(DT_y_pred))

                if (max_r2 < DT_r2):
                    max_r2 = DT_r2
                    max_i = i
                    max_j = j
                    max_k = k
                    max_m = m
            
print(max_r2, max_i, max_j, max_k, max_m)

From the results above the parameters are applied to the model

In [39]:
DT_reg = DecisionTreeRegressor(max_depth=12, max_features=11, 
                      random_state=55, max_leaf_nodes=48).fit(X_train, y_train)

DT_y_pred = DT_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    DT_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(DT_y_pred).loc[:, i])
    DT_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(DT_y_pred).loc[:, i])
    print(outputs[i], 'r2:', DT_r2, '; MSE:', DT_MSE )

diameter: r2: 0.6817126348319635 ; MSE: 0.8371047658297387
Abs: r2: 0.8705969089841237 ; MSE: 780.400112252983
PL: r2: 0.7282100418689819 ; MSE: 581.5228185660071


# AdaBoost Regressor

In [ ]:
max_r2 = 0

max_i, max_j, max_k, max_m  = 0, 0, 0, 0

for i in tqdm(range(5, 15)):
    for j in np.arange(0.1, 2.0, 0.02):
        for k in ('linear', 'square', 'exponential'):
            for m in range(40, 60):
                AB_reg = MultiOutputRegressor(AdaBoostRegressor(n_estimators=i, 
                                                                learning_rate=j, 
                                                                loss=k, 
                                                                random_state=m)).fit(X_train, y_train)

                AB_y_pred = AB_reg.predict(X_test)

                AB_r2 = r2_score(y_test, pd.DataFrame(AB_y_pred))

                if (max_r2 < AB_r2):
                    max_r2 = AB_r2
                    max_i = i
                    max_j = j
                    max_k = k
                    max_m = m
            
print(max_r2, max_i, max_j, max_k, max_m)

From the results above (take a really long time to run) the parameters are applied to the model

In [38]:
AB_reg = MultiOutputRegressor(AdaBoostRegressor(n_estimators=12 ,
                                                learning_rate=0.34, 
                                                loss='exponential', 
                                                random_state=47)).fit(X_train, y_train)

AB_y_pred = AB_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    AB_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(AB_y_pred).loc[:, i])
    AB_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(AB_y_pred).loc[:, i])
    print(outputs[i], 'r2:', AB_r2, '; MSE:', AB_MSE )

diameter: r2: 0.6610728796966749 ; MSE: 0.8913879051563259
Abs: r2: 0.7643858313580297 ; MSE: 1420.9345558370596
PL: r2: 0.6881234053222302 ; MSE: 667.2923371744201


# Ridge

In [26]:
max_r2 = 0

max_i, max_j  = 0, 0

for i in tqdm(np.arange(0.04, 1.4, 0.02)):
    for j in range(30, 60):
        R_reg =  Ridge(alpha=i,
                       random_state=j).fit(X_train, y_train)
            
            
        R_y_pred = R_reg.predict(X_test)
            
        R_r2 = r2_score(y_test, pd.DataFrame(R_y_pred))
            
        if (max_r2 < R_r2):
                max_r2 = R_r2
                max_i = i
                max_j = j
            
print(max_r2, max_i, max_j)

100%|██████████| 68/68 [00:21<00:00,  3.12it/s]

0.505452118088583 0.06 30


From the results above the parameters are applied to the model

In [37]:
R_reg = Ridge(alpha=0.06, 
              random_state=30).fit(X_train, y_train)

R_y_pred = R_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    R_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(R_y_pred).loc[:, i])
    R_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(R_y_pred).loc[:, i])
    print(outputs[i], 'r2:', R_r2, '; MSE:', R_MSE )

diameter: r2: 0.3621857233901635 ; MSE: 1.677469573391533
Abs: r2: 0.6618415887061528 ; MSE: 2039.3551657945416
PL: r2: 0.49232904216943263 ; MSE: 1086.2146943612327


# Ridge Cross Validation 

In [34]:
max_r2 = 0

max_i = 0

for i in tqdm(np.arange(0.01, 10, 0.01)):

        RCV_reg =  RidgeCV(alphas=i).fit(X_train, y_train)


        RCV_y_pred = RCV_reg.predict(X_test)

        RCV_r2 = r2_score(y_test, pd.DataFrame(RCV_y_pred))

        if (max_r2 < RCV_r2):
                max_r2 = RCV_r2
                max_i = i
                max_j = j

print(max_r2, max_i)

100%|██████████| 999/999 [00:10<00:00, 92.69it/s] 

0.5058675808968 0.06999999999999999


From the results above the parameters are applied to the model

In [36]:
RCV_reg = RidgeCV(alphas=0.07).fit(X_train, y_train)

RCV_y_pred = RCV_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    RCV_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(RCV_y_pred).loc[:, i])
    RCV_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(RCV_y_pred).loc[:, i])
    print(outputs[i], 'r2:', RCV_r2, '; MSE:', RCV_MSE )

diameter: r2: 0.36135982901649133 ; MSE: 1.6796416989357186
Abs: r2: 0.6627630834754044 ; MSE: 2033.797843973874
PL: r2: 0.4934798301984733 ; MSE: 1083.7524639579403


# K Neighbors

In [41]:
max_r2 = 0

max_i = 0

max_j = 0

for i in tqdm(range(1, 30)):
    for j in range(1, 20):

        KNN_reg = KNeighborsRegressor(n_neighbors=i, 
                                      p=j).fit(X_train, y_train)

        KNN_y_pred = KNN_reg.predict(X_test)

        KNN_r2 = r2_score(y_test, pd.DataFrame(KNN_y_pred))

        if (max_r2 < KNN_r2):
                max_r2 = KNN_r2
                max_i = i
                max_j = j

print(max_r2, max_i, max_j)

100%|██████████| 29/29 [00:09<00:00,  3.08it/s]

0.7228947186587202 2 4


From the results above the parameters are applied to the model

In [42]:
KNN_reg = KNeighborsRegressor(n_neighbors=2, 
                              p=4).fit(X_train, y_train)

KNN_y_pred = KNN_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    KNN_r2 = r2_score(y_test.iloc[:, i], pd.DataFrame(KNN_y_pred).loc[:, i])
    KNN_MSE = mean_squared_error(y_test.iloc[:, i], pd.DataFrame(KNN_y_pred).loc[:, i])
    print(outputs[i], 'r2:', KNN_r2, '; MSE:', KNN_MSE )

diameter: r2: 0.7489402500096387 ; MSE: 0.660294237925532
Abs: r2: 0.8015202037625659 ; MSE: 1196.9857446808512
PL: r2: 0.6182237022039561 ; MSE: 816.8500053597587
